In [1]:
import mysql.connector
from mysql.connector import errorcode

from sql_formatter.core import format_sql

from databaseversioncontrol.database import Server

from secrets_words import reporting_server_password

In [2]:
connection_config = {
    "user": "sam",
    "password": reporting_server_password,
    "host": "10.0.1.141",
    "port": "3306",
    "raise_on_warnings": True,
}

In [3]:
server = Server(connection_config)

In [4]:
print(server)

Server: <mysql.connector.connection_cext.CMySQLConnection object at 0x0000021CB3D8D940>


In [6]:
databases = server.databases()

In [7]:
[print(d) for d in databases]

Database: Dashboard
Database: Feeds
Database: Keepa
Database: PIM
Database: Reports
Database: Test
Database: URData
Database: msm


[None, None, None, None, None, None, None, None]

In [4]:
try:
    cnx = mysql.connector.connect(**connection_config)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    else:
        print(err)


In [5]:
cursor = cnx.cursor()

In [6]:
cursor.execute("SHOW DATABASES")

databases = [row[0] for row in cursor]

for i in ["information_schema", "mysql", "sys", "performance_schema"]:
    
    databases.remove(i)
    
databases

['Dashboard', 'Feeds', 'Keepa', 'PIM', 'Reports', 'Test', 'URData', 'msm']

In [7]:
tables = {}

for database in databases:

    cursor.execute(f"USE {database}")
    cursor.execute("show full tables where Table_type like 'BASE TABLE';")
    
    tables[database] = [row[0] for row in cursor]
    
tables

{'Dashboard': ['UserLog'],
 'Feeds': ['Alchemy',
  'Eurostar',
  'IM',
  'Likewize',
  'Mazuma',
  'Newway',
  'PCS',
  'SKTes',
  'SkuMatch',
  'TechTiger',
  'WhiteHawk'],
 'Keepa': ['Price'],
 'PIM': ['BBTI', 'MPN', 'Sku'],
 'Reports': ['BBTINewReport',
  'RPT001_BBTI_VIABILITY',
  'RPT002_REFURB_SALES',
  'RPT012_GP_CALCULATOR',
  'RPT014_BBTI_PRICING_CALCULATOR',
  'RPT070_REFURB_PROFITIBILITY'],
 'Test': ['TestPricing', 'teststock'],
 'URData': ['BBTICompetitorOffers',
  'BBTICompetitorOffersLatestTable',
  'BBTIOffers',
  'BBTIOffersLatestTable',
  'Eurostar',
  'PIM',
  'Pricing',
  'PricingLatestTable',
  'Sales',
  'Stock',
  'StockImport',
  'StockLatestTable',
  'Tests'],
 'msm': ['Devices', 'Offers', 'OffersCurrent']}

In [7]:
table_definitions = {}

for database in databases:
    
    table_definitions[database] = {}
    
    for table in tables[database]:
        cursor.execute(f"show create table {database}.{table}")
        
        table_definitions[database][table] = list(cursor)[0][1]

In [8]:
table_definitions["URData"]["PIM"]

'CREATE TABLE `PIM` (\n  `Asin` varchar(10) DEFAULT NULL,\n  `Sku` varchar(40) DEFAULT NULL,\n  `BBTISku` varchar(40) DEFAULT NULL,\n  `DeviceID` int DEFAULT NULL,\n  `BBTIMake` varchar(100) DEFAULT NULL,\n  `BBTIModel` varchar(100) DEFAULT NULL,\n  `BBTICapacity` varchar(100) DEFAULT NULL,\n  `ItemId` int DEFAULT NULL,\n  `Title` varchar(255) DEFAULT NULL,\n  `DeviceType` varchar(100) DEFAULT NULL,\n  `Make` varchar(100) DEFAULT NULL,\n  `Model` varchar(100) DEFAULT NULL,\n  `Capacity` varchar(100) DEFAULT NULL,\n  `Colour` varchar(100) DEFAULT NULL,\n  `URColour` varchar(100) DEFAULT NULL,\n  `XMatch` json DEFAULT NULL,\n  `Updated` datetime DEFAULT NULL,\n  KEY `SKU` (`Sku`),\n  KEY `BBTI` (`BBTISku`,`DeviceID`,`BBTIMake`,`BBTIModel`,`BBTICapacity`,`DeviceType`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci'

In [9]:
views = {}

for database in databases:
    cursor.execute(f"USE {database}")
    cursor.execute("show full tables where Table_type like 'VIEW';")

    views[database] = [row[0] for row in cursor]


In [10]:
views_definition = {}

for database in databases:
    views_definition[database] = {}

    for view in views[database]:
        cursor.execute(f"show create view {database}.{view}")

        views_definition[database][view] = list(cursor)[0][1]

In [11]:
views_definition["Reports"]["BBTIData"]

"CREATE ALGORITHM=UNDEFINED DEFINER=`sam`@`%` SQL SECURITY DEFINER VIEW `Reports`.`BBTIData` AS select `skudata`.`BBTISKu` AS `BBTISku`,`skudata`.`DeviceType` AS `DeviceType`,`skudata`.`BBTIMake` AS `Make`,`skudata`.`BBTIModel` AS `Model`,`skudata`.`BBTICapacity` AS `Variant`,`skudata`.`WMBBAvg7` AS `WMBBAvg7`,`skudata`.`WMUsedAvg7` AS `WMUsedAvg7`,`grade_spread`.`APLUSPercent` AS `APlusPercent`,`grade_spread`.`APercent` AS `APercent`,`grade_spread`.`BPercent` AS `BPercent`,`grade_spread`.`CPercent` AS `CPercent`,`grade_spread`.`DPercent` AS `DPercent`,`grade_spread`.`UPercent` AS `UPercent`,`bbti`.`MeanRepairCost` AS `MeanRepairCost`,`bbti`.`MeanRepairDeduction` AS `MeanRepairDeduction`,`bbti`.`MeanTotalGP` AS `MeanTotalGP`,`offers`.`Price` AS `URBBTIOffer`,`offers`.`MaxPrice` AS `URMaxOffer`,`compoffers`.`BestPrice` AS `BestBBTIOffer`,`skudata`.`AverageCost` AS `MeanSupplierPrice`,`skudata`.`ABpc` AS `SupplierGradePC`,`skudata`.`FORSALETotal` AS `ForSale`,`skudata`.`INBOUNDTotal` AS 

In [12]:
cnx.close()

In [13]:
import os
import shutil
from pathlib import Path

In [14]:
path = Path("/Users/SamuelLloyd/OneDrive - Buyur Ltd/Documents/reportingdatabase/sql_scripts/")

latest_path = path / "Latest"
old_path = path / "Old"


# Delete old files
shutil.rmtree(latest_path)
    
# Create New Latest


In [ ]:
for database, tables in table_definitions.items():
    database_path = latest_path / database
    database_path.mkdir(parents=True, exist_ok=True)
    

    (database_path / "Tables.sql").write_text(
        ";\n".join(
            [table_definition for table_name, table_definition in tables.items()]
        )
    )
    
    
for database, views in views_definition.items():
    database_path = latest_path / database
    database_path.mkdir(parents=True, exist_ok=True)

    (database_path / "Views.sql").write_text(
        ";\n".join([format_sql(view_definition) for view_name, view_definition in views.items()])
    )

    
    

: 